<a href="https://colab.research.google.com/github/HofstraDoboli/TextMining/blob/main/bert_sentiment_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets --upgrade
!pip install transformers[torch]
!pip install evaluate
! pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/TextMining/DataSets/Covid_Tweets/
%ls *.csv

/content/drive/MyDrive/TextMining/DataSets/Covid_Tweets
Corona_NLP_test.csv  Corona_NLP_train.csv


In [22]:
from datasets import load_dataset
# download the data from": https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification
dataset = load_dataset('csv', data_files={'train': 'Corona_NLP_train.csv', 'test': 'Corona_NLP_test.csv'}, encoding = "ISO-8859-1")

In [5]:
# documentation datasets https://huggingface.co/docs/datasets/index
dataset


DatasetDict({
    train: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment'],
        num_rows: 41157
    })
    test: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment'],
        num_rows: 3798
    })
})

In [8]:
dataset['train'][:5] # ['OriginalTweet']

{'UserName': [3799, 3800, 3801, 3802, 3803],
 'ScreenName': [48751, 48752, 48753, 48754, 48755],
 'Location': ['London', 'UK', 'Vagabonds', None, None],
 'TweetAt': ['16-03-2020',
  '16-03-2020',
  '16-03-2020',
  '16-03-2020',
  '16-03-2020'],
 'OriginalTweet': ['@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8',
  'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order',
  'Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P',
  "My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #co

In [23]:
#  Tokenize data
from transformers import AutoTokenizer
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
PYTORCH_NO_CUDA_MEMORY_CACHING=1
# PYTORCH_CUDA_ALLOC_CONF = max_split_size_mb:128
base_model = 'distilbert-base-cased' # "bert-base-cased" # distilbert-base-cased - smaller model
tokenizer = AutoTokenizer.from_pretrained(base_model) # cased = means it was trained to recognize capitalization (vs. bert-base-uncased)

In [24]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [25]:
# exaample what tokenization does
print(dataset['train'][10]["OriginalTweet"])
print(len(dataset['train'][10]["OriginalTweet"].split(' ')))
test_token = tokenizer(dataset['train'][10]["OriginalTweet"])
print(test_token.keys())
print(len(test_token['input_ids']))
print(test_token['input_ids'])

# input_ids = token's ids for parts of words in the text
# token_type_ids = all 0's if the input is one texr, 0's followed by 1's if input is a two sentence text
# attention_mask = if any of the words should be masked

All month there hasn't been crowding in the supermarkets or restaurants, however reducing all the hours and closing the malls means everyone is now using the same entrance and dependent on a single supermarket. #manila #lockdown #covid2019 #Philippines https://t.co/HxWs9LAnF9
39
dict_keys(['input_ids', 'attention_mask'])
73
[101, 1398, 2370, 1175, 8186, 112, 189, 1151, 3515, 1158, 1107, 1103, 20247, 1116, 1137, 7724, 117, 1649, 7914, 1155, 1103, 2005, 1105, 5134, 1103, 8796, 1116, 2086, 2490, 1110, 1208, 1606, 1103, 1269, 3448, 1105, 7449, 1113, 170, 1423, 20247, 119, 108, 1299, 8009, 108, 5842, 5455, 108, 1884, 18312, 10973, 16382, 108, 4336, 18630, 131, 120, 120, 189, 119, 1884, 120, 145, 1775, 2924, 1116, 1580, 10783, 1179, 2271, 1580, 102]


In [26]:
# tokenize all dataset
def tokenize_data(example):
    return tokenizer(example['OriginalTweet'], padding='max_length',
                     truncation = True, return_tensors = "pt").to(device)

dataset = dataset.map(tokenize_data, batched = True)  # tokenizes all the data

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [ ]:
print(type(dataset['train'][1]['input_ids'])) # attention_mask is 1 for non padding tokens only, 0 for padding tokens
print(len(dataset['train'][1]['input_ids']))

<class 'list'>
512


In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment', 'input_ids', 'attention_mask'],
        num_rows: 41157
    })
    test: Dataset({
        features: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment', 'input_ids', 'attention_mask'],
        num_rows: 3798
    })
})

In [13]:
# transform the labels from words to numbers
def labels2num(label):

    label = label['Sentiment']
    dict_labels = {'Positive':0, 'Negative':1, 'Neutral':2,
                   'Extremely Positive':3, 'Extremely Negative':4}
    return {'labels': dict_labels[label]}

print(labels2num(dataset['train'][1]))

{'labels': 0}


In [28]:
remove_columns = ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
dataset = dataset.map(labels2num, remove_columns=remove_columns)

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [29]:
dataset # see it added 'labels' feature and removed the rest

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 41157
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3798
    })
})

In [ ]:
len(dataset['train']['input_ids'][1:3])

2

In [30]:
# set format of the dataset to torch
dataset.set_format(type="torch", columns=["input_ids",  "attention_mask", "labels"]) # "token_type_ids",
dataset['train'].format['type']

'torch'

In [31]:
# load training arguments, trainer, AutoModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
#data_collator = DataCollatorWithPadding(tokenizer) # use only if you did not pad the data with the tokenizer
# base_model = "bert-base-cased" # already defined above
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels = 5).to(device)


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
print(model.parameters)

<bound method Module.parameters of DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropo

In [33]:


training_args = TrainingArguments(
    output_dir  = 'OutputTrain', # output directory
    num_train_epochs = 3,
    eval_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    weight_decay = 0.01,
    push_to_hub = False,
    report_to = None
)
# training_args
import wandb # a library to visualize training process and results - disable it
wandb.init(mode='disabled')

In [34]:
# set the training and validation datesets
train_dataset = dataset['train'].shuffle(seed=10).select(range(40000))
eval_dataset  = dataset['train'].shuffle(seed=10).select(range(40000, 41000)) # validation dataset

In [ ]:
train_dataset[1:3]['labels'].shape # 'input_ids'

torch.Size([2])

In [35]:
# evaluate the model
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {}
    metrics.update(accuracy.compute(predictions=predictions, references=labels))
    metrics.update(precision.compute(predictions=predictions, references=labels, average='weighted'))
    metrics.update(recall.compute(predictions=predictions, references=labels, average='weighted'))
    metrics.update(f1.compute(predictions=predictions, references=labels, average='weighted'))
    return metrics


In [36]:
trainer = Trainer(
    model= model,
    args = training_args,
    train_dataset= train_dataset,
    eval_dataset = eval_dataset,
    #data_collator=data_collator, # only if you want to pad inputs during training
    compute_metrics = compute_metrics
    )



In [37]:
# test compute_metrics
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


{'eval_loss': 1.597570538520813,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': 0.197,
 'eval_precision': 0.1288306375616311,
 'eval_recall': 0.197,
 'eval_f1': 0.1256483436853002,
 'eval_runtime': 14.8676,
 'eval_samples_per_second': 67.26,
 'eval_steps_per_second': 2.152}

In [31]:
!pip install pynvml
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.0 MB/s eta 0:00:00
GPU memory occupied: 2481 MB.


In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Precision,Recall,F1
1,0.795000,0.658665,0.001600,0.757000,0.758286,0.757000,0.756151
2,0.509300,0.525282,0.001600,0.804000,0.805657,0.804000,0.804266
3,0.411200,0.539268,0.001600,0.805000,0.808933,0.805000,0.804813


TrainOutput(global_step=3750, training_loss=0.6211861633300781, metrics={'train_runtime': 5372.0958, 'train_samples_per_second': 22.338, 'train_steps_per_second': 0.698, 'total_flos': 1.58969382912e+16, 'train_loss': 0.6211861633300781, 'epoch': 3.0})

In [39]:
# check results of the trained model
trainer.evaluate(dataset['test'])

{'eval_loss': 0.5950799584388733,
 'eval_model_preparation_time': 0.0016,
 'eval_accuracy': 0.7880463401790416,
 'eval_precision': 0.7906534910109877,
 'eval_recall': 0.7880463401790416,
 'eval_f1': 0.788185642690859,
 'eval_runtime': 55.8437,
 'eval_samples_per_second': 68.011,
 'eval_steps_per_second': 2.131,
 'epoch': 3.0}

In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return model.config.id2label[predicted_class_id]

In [ ]:
trainer.train(sume_from_checkpoint = "OutputTrain/checkpoint-3500")

Loading model from OutputTrain/checkpoint-3500.
***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 108314117
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 3500
  Will skip the first 2 epochs then the first 1000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
3,0.310200,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32


Trainer is attempting to log a value of "{'accuracy': 0.852}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.857585957253941}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8567195710330532}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8597792422199729}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3750, training_loss=0.01783670450846354, metrics={'train_runtime': 200.9568, 'train_samples_per_second': 597.143, 'train_steps_per_second': 18.661, 'total_flos': 3.157417709568e+16, 'train_loss': 0.01783670450846354, 'epoch': 3.0})

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.261200,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}
2,0.268300,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}
3,0.264100,0.440796,{'accuracy': 0.852},{'f1': 0.857585957253941},{'precision': 0.8567195710330532},{'recall': 0.8597792422199729}


Saving model checkpoint to OutputTrain/checkpoint-500
Configuration saved in OutputTrain/checkpoint-500/config.json
Model weights saved in OutputTrain/checkpoint-500/pytorch_model.bin
Saving model checkpoint to OutputTrain/checkpoint-1000
Configuration saved in OutputTrain/checkpoint-1000/config.json
Model weights saved in OutputTrain/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 32
Trainer is attempting to log a value of "{'accuracy': 0.852}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.857585957253941}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8567195710330532}" of type <class 'dict'> for key "eval/precision

TrainOutput(global_step=3750, training_loss=0.26746157023111977, metrics={'train_runtime': 2776.7222, 'train_samples_per_second': 43.216, 'train_steps_per_second': 1.351, 'total_flos': 3.157417709568e+16, 'train_loss': 0.26746157023111977, 'epoch': 3.0})